# Multi-Agent Collaboration with LangGraph and ChatAmazonNova

This notebook demonstrates multiple specialized agents working together under a supervisor.

## Architecture
- **Supervisor**: Routes tasks to appropriate agents
- **Writer**: Generates creative content
- **Translator**: Translates text between languages
- **Critic**: Reviews and improves content

## Key Concepts
- Agent specialization
- Supervisor pattern
- Conditional routing between agents

In [ ]:
%env NOVA_API_KEY=<YOUR-API-KEY>
%env NOVA_BASE_URL=https://api.nova.amazon.com/v1/

In [ ]:
%pip install -e .

## Setup and Imports

In [ ]:
from typing import TypedDict

from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import StateGraph, START, END

from langchain_amazon_nova import ChatAmazonNova

## Define State

State tracks messages, the next agent to call, and the original task.

In [ ]:
class AgentState(TypedDict):
    """State passed between agents."""
    messages: list
    next_agent: str
    task: str

## Create Specialized Agents

Each agent has a specific role and system prompt.

In [ ]:
def create_writer_agent(llm):
    """Create a writer agent."""
    system_message = SystemMessage(
        content="""You are a creative writer. Your task is to write engaging, 
        well-structured content based on the user's request. Focus on clarity, 
        creativity, and proper structure."""
    )
    
    def writer(state: AgentState) -> dict:
        messages = [system_message] + state["messages"]
        response = llm.invoke(messages)
        return {
            "messages": state["messages"] + [response],
            "next_agent": "supervisor"
        }
    
    return writer


def create_translator_agent(llm):
    """Create a translator agent."""
    system_message = SystemMessage(
        content="""You are an expert translator. Translate the provided text 
        accurately while preserving the original meaning, tone, and style. 
        Only output the translation, no explanations."""
    )
    
    def translator(state: AgentState) -> dict:
        messages = [system_message] + state["messages"]
        response = llm.invoke(messages)
        return {
            "messages": state["messages"] + [response],
            "next_agent": "supervisor"
        }
    
    return translator


def create_critic_agent(llm):
    """Create a critic agent."""
    system_message = SystemMessage(
        content="""You are a thoughtful critic and editor. Review the provided 
        content and suggest specific improvements for clarity, accuracy, and 
        engagement. Be constructive and specific."""
    )
    
    def critic(state: AgentState) -> dict:
        messages = [system_message] + state["messages"]
        response = llm.invoke(messages)
        return {
            "messages": state["messages"] + [response],
            "next_agent": "supervisor"
        }
    
    return critic


print("Specialized agents defined!")

## Create Supervisor

The supervisor decides which agent should handle each task.

In [ ]:
def create_supervisor(llm):
    """Create a supervisor that routes tasks."""
    
    system_message = SystemMessage(
        content="""You are a supervisor managing: writer, translator, critic.
        
        Based on the conversation, decide which agent acts next, or if done.
        
        - 'writer' for creative writing or content generation
        - 'translator' for translation tasks
        - 'critic' for reviewing or improving content
        - 'finish' when the task is complete
        
        Respond with ONE WORD: writer, translator, critic, or finish."""
    )
    
    def supervisor_node(state: AgentState) -> dict:
        messages = [system_message] + state["messages"]
        response = llm.invoke(messages)
        next_agent = response.content.strip().lower()
        
        # Validate response
        if next_agent not in ["writer", "translator", "critic", "finish"]:
            next_agent = "finish"
        
        return {"next_agent": next_agent}
    
    return supervisor_node

## Build the Multi-Agent Graph

In [ ]:
def create_multi_agent_graph(llm):
    """Create the multi-agent graph."""
    
    writer = create_writer_agent(llm)
    translator = create_translator_agent(llm)
    critic = create_critic_agent(llm)
    supervisor = create_supervisor(llm)
    
    workflow = StateGraph(AgentState)
    
    # Add nodes
    workflow.add_node("supervisor", supervisor)
    workflow.add_node("writer", writer)
    workflow.add_node("translator", translator)
    workflow.add_node("critic", critic)
    
    # Set entry point
    workflow.add_edge(START, "supervisor")
    
    # Routing function
    def route(state: AgentState) -> str:
        return state["next_agent"]
    
    # Add conditional routing from supervisor
    workflow.add_conditional_edges(
        "supervisor",
        route,
        {
            "writer": "writer",
            "translator": "translator",
            "critic": "critic",
            "finish": END,
        }
    )
    
    # All agents return to supervisor
    workflow.add_edge("writer", "supervisor")
    workflow.add_edge("translator", "supervisor")
    workflow.add_edge("critic", "supervisor")
    
    return workflow.compile()

## Initialize the System

In [ ]:
# Initialize model
llm = ChatAmazonNova(
    model="nova-pro-v1",
    temperature=0.7,
)

# Create multi-agent system
agent_system = create_multi_agent_graph(llm)

print("Multi-agent system initialized!")

## Example 1: Creative Writing

In [ ]:
task = "Write a short haiku about autumn leaves"

result = agent_system.invoke({
    "messages": [HumanMessage(content=task)],
    "next_agent": "supervisor",
    "task": task
})

print(f"Task: {task}\n")
print("Result:")
print(result["messages"][-1].content)

## Example 2: Write and Translate

In [ ]:
task = "Write a short poem about the ocean and then translate it to Spanish"

result = agent_system.invoke({
    "messages": [HumanMessage(content=task)],
    "next_agent": "supervisor",
    "task": task
})

print(f"Task: {task}\n")
print("Result:")
print(result["messages"][-1].content)

## Example 3: Write and Critique

In [ ]:
task = "Write a product description for a smartwatch and have it reviewed"

result = agent_system.invoke({
    "messages": [HumanMessage(content=task)],
    "next_agent": "supervisor",
    "task": task
})

print(f"Task: {task}\n")
print("Result:")
print(result["messages"][-1].content)

## View Collaboration Flow

See which agents participated in the task.

In [ ]:
print(f"Total messages: {len(result['messages'])}\n")
print("Agent collaboration flow:\n")

for i, msg in enumerate(result['messages']):
    if msg.type == "human":
        print(f"{i+1}. User: {msg.content[:80]}...")
    elif msg.type == "ai":
        content_preview = msg.content[:80] if len(msg.content) > 80 else msg.content
        print(f"{i+1}. Agent: {content_preview}...")
    print()

## Try Your Own Task

In [ ]:
# Modify the task below
your_task = "Write a limerick about coding and translate it to French"

result = agent_system.invoke({
    "messages": [HumanMessage(content=your_task)],
    "next_agent": "supervisor",
    "task": your_task
})

print(f"Task: {your_task}\n")
print("Result:")
print(result["messages"][-1].content)